## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-23-08-54-53 +0000,bbc,Three West African countries to quit Internati...,https://www.bbc.com/news/articles/czjvp0pr3eko...
1,2025-09-23-08-50-00 +0000,wsj,European Business Activity Inches Up But Impet...,https://www.wsj.com/economy/eurozone-business-...
2,2025-09-23-08-47-14 +0000,nyt,France and Other Nations Recognize Palestinian...,https://www.nytimes.com/live/2025/09/22/world/...
3,2025-09-23-08-46-01 +0000,nyt,Drone Sightings Force Copenhagen and Oslo Airp...,https://www.nytimes.com/2025/09/22/world/europ...
4,2025-09-23-08-42-00 +0000,wsj,"In Rare Visit to China, U.S. Lawmakers Push fo...",https://www.wsj.com/world/china/in-rare-visit-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2339/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
45,trump,61
415,kirk,26
357,kimmel,20
414,charlie,19
356,jimmy,17


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
227,2025-09-22-18-27-02 +0000,nypost,"Elon Musk, President Trump come together at Ch...",https://nypost.com/2025/09/22/us-news/elon-mus...,145
321,2025-09-22-11-14-50 +0000,nypost,Charlie Kirk inspired Trump and Elon Musk to r...,https://nypost.com/2025/09/22/us-news/charlie-...,132
198,2025-09-22-19-59-40 +0000,nypost,Trump open to national holiday honoring Charli...,https://nypost.com/2025/09/22/us-news/trump-li...,126
317,2025-09-22-11-42-53 +0000,bbc,Trump hails Charlie Kirk as martyr to thousand...,https://www.bbc.com/news/articles/ckgee0x9p40o...,120
52,2025-09-23-03-02-00 +0000,wsj,In labeling Tylenol use in pregnant women as a...,https://www.wsj.com/politics/in-targeting-a-co...,117


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
227,145,2025-09-22-18-27-02 +0000,nypost,"Elon Musk, President Trump come together at Ch...",https://nypost.com/2025/09/22/us-news/elon-mus...
271,78,2025-09-22-15-31-23 +0000,nypost,Howard Stern blasts ABC suspension of Jimmy Ki...,https://nypost.com/2025/09/22/media/howard-ste...
52,56,2025-09-23-03-02-00 +0000,wsj,In labeling Tylenol use in pregnant women as a...,https://www.wsj.com/politics/in-targeting-a-co...
322,45,2025-09-22-11-11-45 +0000,bbc,UK says Israel must not retaliate against Pale...,https://www.bbc.com/news/articles/c1wggrdn9dno...
269,42,2025-09-22-15-36-02 +0000,cbc,"Ford government lays out demands for bail, sen...",https://www.cbc.ca/news/canada/toronto/ontario...
310,41,2025-09-22-12-55-13 +0000,nypost,Putin says he is open to one-year extension of...,https://nypost.com/2025/09/22/world-news/putin...
225,37,2025-09-22-18-28-40 +0000,nypost,Russia risks ‘expansion’ of war with intrusion...,https://nypost.com/2025/09/22/us-news/russia-r...
238,34,2025-09-22-17-59-22 +0000,nyt,American Investors Will License and Oversee Ti...,https://www.nytimes.com/2025/09/22/us/politics...
306,33,2025-09-22-13-00-00 +0000,wsj,Atlanta Fed President Raphael Bostic said infl...,https://www.wsj.com/economy/central-banking/fe...
75,30,2025-09-23-01-00-00 +0000,wsj,The worker visas that President Trump says now...,https://www.wsj.com/economy/jobs/h1b-visas-wor...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
